# Spatial Tx: ingest 10x Visium sample data
- Download sample data from https://cf.10xgenomics.com/samples/spatial-exp/2.1.0/CytAssist_FFPE_Protein_Expression_Human_Glioblastoma
- Ingest into spatial SOMA
- Read from SOMA, inspect/plot data

**WARNING** _Spatial support is experimental and under active development. There will likely be breaking changes to both the storage format and API._

In [1]:
from functools import partial
import numpy as np
from os import makedirs, remove
from os.path import basename, exists, join
import shlex
from shutil import rmtree
from subprocess import check_call
from sys import stderr
from urllib.parse import urlparse
err = partial(print, file=stderr)

import scanpy as sc

from tiledbsoma.experimental import from_visium
from tiledbsoma import Experiment
import tiledbsoma

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mplp
from matplotlib.collections import PatchCollection

[Papermill](https://papermill.readthedocs.io/en/latest/) parameters:

In [2]:
# 10x visium sample data paths to download; will be skipped if already present locally, unless `overwrite`
url_base = 'https://cf.10xgenomics.com/samples/spatial-exp/2.1.0'
dataset_name = 'CytAssist_FFPE_Protein_Expression_Human_Glioblastoma'
filtered_h5 = 'CytAssist_FFPE_Protein_Expression_Human_Glioblastoma_filtered_feature_bc_matrix.h5'
spatial_tar = 'CytAssist_FFPE_Protein_Expression_Human_Glioblastoma_spatial.tar.gz'

# Local download/ingestion paths/configs:
data_root = 'data'     # Sync 10x Visium spatial data into f"{data_root}/{dataset_name}/10x". By default, spatial SOMA data will be ingested into f"{data_root}/{dataset_name}/soma" as well.
exp_uri = None         # Ingest spatial SOMA data here; defaults to f"{data_root}/{dataset_name}/soma"
scene_name = "scene1"  # Scene name to write, in ingested spatial SOMA
overwrite_10x = False  # If  already exists, remove and re-ingest it
overwrite_exp = False  # If `exp_uri` already exists, remove and re-ingest it

In [3]:
# Set default paths
dataset_root = join(data_root, dataset_name)
data_dir_10x = join(dataset_root, '10x')
if exp_uri is None:
    exp_uri = join(dataset_root, 'soma')
exp_uri

'data/CytAssist_FFPE_Protein_Expression_Human_Glioblastoma/soma'

In [4]:
def sh(*cmd):
    err(f"Running: {shlex.join(cmd)}")
    check_call(cmd)    

## Download sample data from 10x
This section will download data from 10x and use that data to generate the TileDB-SOMA `Experiment` with spatial data.

In [5]:
if exists(exp_uri):
    if overwrite_exp:
        err(f"Removing {exp_uri}")
        rmtree(exp_uri)

In [6]:
paths = {
    filtered_h5: 'filtered_feature_bc_matrix.h5',
    spatial_tar: spatial_tar
}
if exists(exp_uri):
    err(f"{exp_uri} exists; skipping 10x data download")
else:
    for src_name, dst_name in paths.items():
        src = f'{url_base}/{dataset_name}/{src_name}'
        dst = join(data_dir_10x, dst_name)
        if exists(dst):
            if overwrite_10x:
                err(f"{dst} exists, removing")
                remove(dst)
        if not exists(dst):
            makedirs(data_dir_10x, exist_ok=True)
            sh('wget', '-qO', dst, src)
            if dst.endswith('.tar.gz'):
                sh('tar', '-C', data_dir_10x, '-xvf', dst)

In [ ]:
if not exists(exp_uri):
    err(f"Ingesting {data_dir_10x} to {exp_uri}")
    from_visium(
        exp_uri,
        input_path=data_dir_10x,
        measurement_name="RNA",
        scene_name=scene_name,
        use_raw_counts=False,
    )

Ingesting data/CytAssist_FFPE_Protein_Expression_Human_Glioblastoma/10x to data/CytAssist_FFPE_Protein_Expression_Human_Glioblastoma/soma
/home/julia/.pyenv/versions/3.11.8/envs/soma/lib/python3.11/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/julia/.pyenv/versions/3.11.8/envs/soma/lib/python3.11/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
<frozen abc>:119: FutureWarning: SparseDataset is deprecated and will be removed in late 2024. It has been replaced by the public classes CSRDataset and CSCDataset.

For instance checks, use `isinstance(X, (anndata.experimental.CSRDataset, anndata.experimental.CSCDataset))` instead.

For creation, use `anndata.experimental.sparse_dataset(X)` instead.



## Data access

Spatial SOMA experiments can be acess and queries using any of `tiledbsoma`'s exsiting APIs.

In [ ]:
exp = Experiment.open(exp_uri)
exp.spatial

Here, we're loading the non-spatial elements into memory as a standard AnnData object.

In [ ]:
adata = tiledbsoma.io.to_anndata(
    experiment=exp,
    measurement_name="RNA",
    X_layer_name="data",
)

adata

Let's visualize the most highly expressed genes in the dataset.

In [ ]:
sc.pl.highest_expr_genes(adata)

## Spatial components

The new SOMA Experiment contains spatial data for this experiment stored in the spatial property of the Experiment. If we view the spatial collection we there is only one Scene named scene0.

In [ ]:
scene = exp.spatial[scene_name]
scene

The scene contains three folders:

    img - A SOMA Collection that stores imagery data. In this example it contains an image pyramid for the high- and low-resolution slide.
    obsl - A SOMA Collection storing location data in the form of SOMADataFrames defined on the obs somajoinid (or obs_id). In this example, this contains a single dataframe with basic information about the Visium spot locations and sizes.
    varl - A collection that stores location data in form form of dataframes defined on the var somajoinid (or var_id). This collection is nested with the first layer mapping from measurement name to a collection storing dataframes. There is no feature spatial data in the Visium example, so this collection is empty.

Inside the img collection, there is a Image2DCollection storing the slide images. Here we view basic information about the slide zoom levels and read the data in the hires image.

The scene is defined on a pixel coordinate space.

In [ ]:
scene.coordinate_space

## Images
Inside the `img` collection, there is a `MultiscaleImage` storing the slide images. Here we view basic information about the slide zoom levels and read the data in the `hires` image.

In [ ]:
tissue_image = scene.img["tissue"]
tissue_image

In [ ]:
tissue_image.coordinate_space

Use the level_count property to get the number of levels in the `MultiscaleImage`.

In [ ]:
tissue_image.level_count

The `MultiscaleImage` has reference properties that are used to get the scale for the different levels.

In [ ]:
tissue_image.reference_level_properties

Examine the metadata for each image level in the "tissue" collection.

In [ ]:
for level in range(tissue_image.level_count):
    print(f"Level {level}: {tissue_image.level_properties(level)}")

Accessing individual elements of the `MultiscaleImage` collection returns a `DenseNDArray` that can be read using the standard API.

In [ ]:
lowres = tissue_image["lowres"]
lowres

In [ ]:
im = lowres.read().to_numpy()

In [ ]:
plt.imshow(im)

## Spatial Dataframe
The `obsl` collection stores location data in the `loc` dataframe. This dataframes stores the spot locations of the Visium dataset with `soma_joinid` matching those used in the `obs` dataframe in the root `Experiment`.

In [ ]:
spots_point_cloud = scene.obsl["loc"]

In [ ]:
spots_point_cloud.coordinate_space

In [ ]:
spots = spots_point_cloud.read().concat().to_pandas()
spots

In [ ]:
obs_df = exp.obs.read().concat().to_pandas()
obs_df

In [ ]:
joinid_counts = spots.soma_joinid.value_counts().value_counts()
assert len(joinid_counts) == 1
joinid_counts

We take the data from the spot dataframe and create a plot showing the regions in the tissue.

In [ ]:
radius = scene.obsl["loc"].metadata["soma_geometry"]
spot_patches = PatchCollection([
    mplp.Circle((row["x"], row["y"]), radius=radius, color='b')
    for _, row in spots.iterrows()
])

In [ ]:
fig, ax = plt.subplots()
ax.set_xlim([0, 24592])
ax.set_ylim([0, 28202])
ax.invert_yaxis()
ax.add_collection(spot_patches)
plt.show()

Reading a region using coordinate transforms.

## Using Coordinate Spaces and Transforms
Coordinate spaces and coordinate transforms can be used to project both the Visium spots and the same image.

The scene allows us to check the coordinate spaces of individual elements.

In [ ]:
scene.get_transformation_to_multiscale_image("tissue")

In [ ]:
scene.get_transformation_to_multiscale_image("tissue", level=0)

In [ ]:
scene.get_transformation_to_point_cloud("loc")

We want to take a piece of the region that both the multiscale image and point cloud are defined on to do the following:

* Read the region from the highest resolution level of the multiscale image.
* Read the region from the point cloud storing the spot locations.
* Use the transformation from the multiscale image to adjust and output of the point cloud and plot the two items together.

In [ ]:
x_min, x_max = (0, 12000)
y_min, y_max = (14000, 24592)
fullres_region = [x_min, y_min, x_max, y_max]

In [ ]:
hires_read = tissue_image.read_level(0,  fullres_region)
hires_read

We take the data from the spot dataframe, convert it to the hires resolution, and plot the two images together.

In [ ]:
point_reader = scene.obsl["loc"].read_region(fullres_region)
point_reader

In [ ]:
spots = point_reader.data.concat().to_pandas()
spots

In [ ]:
spot_to_hires_matrix = hires_read.coordinate_transform.inverse_transform().augmented_matrix
spot_to_hires_matrix

In [ ]:
scale_x = spot_to_hires_matrix[0, 0]
scale_y = spot_to_hires_matrix[1, 1]
offset_x = spot_to_hires_matrix[0, 2]
offset_y = spot_to_hires_matrix[1, 2]

In [ ]:
radius = scene.obsl["loc"].metadata["soma_geometry"]
spot_patches = PatchCollection([
    mplp.Ellipse(
        (scale_x * row["x"] + offset_x, scale_y * row["y"] + offset_y),
        width = radius * scale_x,
        height = radius * scale_y,
        fill=False,
        alpha=0.8,
    )
    for _, row in spots.iterrows()
])

In [ ]:
fig, ax = plt.subplots()
ax.imshow(hires_read.data)
ax.add_collection(spot_patches)

plt.show()